In [0]:
# Gold from Silver (SQL-only, run in PySpark notebook)

# Silver tables
SILVER_CLIENT = "silver_client"
SILVER_PRODUCT = "silver_product"
SILVER_CHANNEL = "silver_channel"
SILVER_MKT_PREF = "silver_consent" #SCD2

# 5) C360_CLIENT_BANKING_SIGNALS (weekly)
# from channel activities aggregate：branch visit、branch txn count、ATM use、call center、fraud report
spark.sql(f"""
CREATE OR REPLACE TABLE c360_client_banking_signals AS
WITH base AS (
SELECT
cast(client_id as bigint) AS client_id,
cast(banking_channel_name as string) AS banking_channel_name,
cast(interaction_type as string) AS interaction_type,
cast(interaction_date as date) AS d
FROM {SILVER_CHANNEL}
),
w AS (
SELECT
client_id,
weekofyear(d) AS iso_week_num,
next_day(d, 'Sun') AS week_end_date,
banking_channel_name,
interaction_type
FROM base
)
SELECT
client_id,
iso_week_num,
week_end_date,
max(CASE WHEN lower(banking_channel_name) = 'branch' THEN 1 ELSE 0 END) AS branch_visit_ind,
sum(CASE WHEN lower(banking_channel_name) = 'branch' THEN 1 ELSE 0 END) AS branch_trxn_cnt,
max(CASE WHEN lower(banking_channel_name) = 'atm' THEN 1 ELSE 0 END) AS atm_use_ind,
max(CASE WHEN lower(banking_channel_name) = 'call center' THEN 1 ELSE 0 END) AS call_center_ind,
max(CASE WHEN lower(interaction_type) LIKE '%fraud%' AND lower(banking_channel_name) = 'call center' THEN 1 ELSE 0 END) AS call_center_fraud_report_ind,
max(CASE WHEN lower(interaction_type) LIKE '%fraud%' AND lower(banking_channel_name) = 'branch' THEN 1 ELSE 0 END) AS branch_fraud_report_ind,
sum(CASE WHEN lower(interaction_type) LIKE '%fraud%' THEN 1 ELSE 0 END) AS num_fraud_report_channel,
current_date() AS proc_dt
FROM w
GROUP BY client_id, iso_week_num, week_end_date
""")

# Quick checks
spark.sql(f"select * from c360_client_banking_signals").display()